In [30]:
import numpy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # activation functions
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm import tqdm

Create Fully Connected Network

In [2]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes): # input_size=28*28=784
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50) # 50 hidden layer nodes
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

### Set Device

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Hyperparameters

In [27]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

### Load Data

In [11]:
train_dataset = datasets.MNIST(root='dataset/', 
                               train=True, 
                               transform=transforms.ToTensor(), 
                               download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', 
                               train=False, 
                               transform=transforms.ToTensor(), 
                               download=True)

test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

### Initialize the Network

In [12]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

# Loss and Optimizer

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Remember to give model parameters here

### Train Model

In [31]:
for epoch in tqdm(range(num_epochs)):
  for ind, (data, target) in enumerate(train_loader):
    data = data.to(device=device)
    data = data.reshape(data.shape[0], -1)
    target = target.to(device=device)
    
    # forward
    scores = model(data)
    loss = criterion(scores, target)

    # backward
    optimizer.zero_grad() # set all the gradients to 0 for each batch
    loss.backward()

    # gradient descent or adam step - updates the parameters
    optimizer.step()

  print(f"Epoch {epoch+1} is done")

 10%|█         | 1/10 [00:08<01:18,  8.71s/it]

Epoch 1 is done


 20%|██        | 2/10 [00:17<01:10,  8.77s/it]

Epoch 2 is done


 30%|███       | 3/10 [00:26<01:01,  8.83s/it]

...

Epoch 8 is done


 90%|█████████ | 9/10 [01:20<00:08,  8.96s/it]

Epoch 9 is done


100%|██████████| 10/10 [01:29<00:00,  8.92s/it]

Epoch 10 is done


# Testing Accuracy

In [44]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval() # Let the model know that this is evaluation mode

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      x = x.reshape(x.shape[0], -1)
      y = y.to(device=device)

      scores = model(x)
      
      # Which class has the max value
      _, pred = scores.max(dim=1)
      num_correct += (pred == y).sum()
      num_samples += pred.size(0) # 64

    print(f"Got {num_correct} / {num_samples} with accuracy {(float(num_correct)/float(num_samples)*100):.2f}%")

In [45]:
check_accuracy(test_loader, model)

Got 9712 / 10000 with accuracy 97.12%
